# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### 1. Implementing the Norvig's SpellChecker as baseline 

In [1]:
import re
from collections import Counter

from typing import List


class NorvigSpellChecker:
    """Norvig's Spell Checker"""

    def __init__(self, path_to_vocab: str = None):
        self.WORDS = None
        self.total_n = None

        if path_to_vocab:
            self.vocab_init(path_to_vocab)

    def vocab_init(self, path_to_vocab: str):
        self.WORDS = Counter(self.words(open(path_to_vocab).read()))
        self.total_n = sum(self.WORDS.values())

    def word_prob(self, word: str, total_n: int = None):
        """Probability of `word`."""

        if total_n is None:
            total_n = self.total_n

        return self.WORDS[word] / total_n

    def correction(self, word: str):
        """Most probable spelling correction for word."""
        return max(self.candidates(word), key=self.word_prob)

    def text_correction(self, text: str):
        """Most probable spelling correction for text."""
        return re.sub(r'\w+', lambda m: self.correction(m.group(0)), text)

    def candidates(self, word: str):
        """Generate possible spelling corrections for word."""
        return self.known(self.edits1(word)) or self.known([word]) or self.known(self.edits2(word)) or [word]

    def known(self, words: List[str]):
        """The subset of `words` that appear in the dictionary of WORDS."""
        return set(w for w in words if w in self.WORDS)

    @staticmethod
    def words(text: str):
        return re.findall(r'\w+', text.lower())

    @staticmethod
    def edits1(word: str):
        """All edits that are one edit away from `word`."""
        letters = 'abcdefghijklmnopqrstuvwxyz'

        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]

        return set(deletes + transposes + replaces + inserts)

    @staticmethod
    def edits2(word: str):
        """All edits that are two edits away from `word`."""
        return (e2 for e1 in NorvigSpellChecker.edits1(word) for e2 in NorvigSpellChecker.edits1(e1))

### 2. Implementing the Context Sensitive Spell Checker

In [2]:
WORDS_FREQ = dict()
EDITS_FREQ = dict()


def add_edit(edits, L, R):
    """Add an edit to the list of edits."""
    return edits + [R + '|' + L]


def generate_edits(head, tail, edit_distance, edits, result, possible_prefixes,
                   alphabet='abcdefghijklmnopqrstuvwxyz'):
    """Generate edits for the given head and tail."""

    C = head + tail
    if C in WORDS_FREQ:
        e = '+'.join(edits)
        result[C] = (max(result.get(C, ('', 0))[0], e,
                         key=(lambda x: EDITS_FREQ.get(x, 1) / sum(EDITS_FREQ.values()))), edit_distance)

    if edit_distance <= 0:
        return

    extensions = [head + c for c in alphabet if head + c in possible_prefixes]
    p = head[-1] if head else '<'

    # Insertion
    for h in extensions:
        generate_edits(h, tail, edit_distance - 1, add_edit(edits, p + h[-1], p), result, possible_prefixes)

    if not tail:
        return

    # Deletion
    generate_edits(head, tail[1:], edit_distance - 1, add_edit(edits, p, p + tail[0]), result, possible_prefixes)

    # Match or Replacement
    for h in extensions:
        if h[-1] == tail[0]:
            generate_edits(h, tail[1:], edit_distance, edits, result, possible_prefixes)
        else:
            generate_edits(h, tail[1:], edit_distance - 1, add_edit(edits, h[-1], tail[0]), result, possible_prefixes)

    # Transpose
    if len(tail) >= 2 and tail[0] != tail[1] and head + tail[1] in possible_prefixes:
        generate_edits(head + tail[1], tail[0] + tail[2:], edit_distance - 1,
                       add_edit(edits, tail[1] + tail[0], tail[0:2]), result,
                       possible_prefixes)


class CSSpellChecker:
    """
    Context Sensitive Spell Checker
    
    The class implements context-sensitive spell checker using N-gram language model.
    It extends the NorvigSpellChecker class.
    """

    def __init__(self, path_to_words_freq: str, path_to_edits_freq: str,
                 path_to_bigrams: str, path_to_fivegrams: str,
                 word_sep: str = ' ', edit_sep: str = ' ',
                 max_edit_distance: int = 2, freq_threshold: int = 1):
        super().__init__()

        self.bigrams_freq = self.load_ngram_data(path_to_bigrams)
        self.first_word_frequencies = self.calculate_first_word_frequencies()

        self.fivegrams_freq = self.load_ngram_data(path_to_fivegrams)

        self.words_freq = self.load_freqs(path_to_words_freq, sep=word_sep)
        self.words_freq = Counter({word: freq for word, freq in self.words_freq.items() if freq > freq_threshold})

        self.edits_freq = self.load_freqs(path_to_edits_freq, sep=edit_sep)

        self.possible_prefixes = self.compute_possible_prefixes()
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
        self.max_edit_distance = max_edit_distance

        global WORDS_FREQ, EDITS_FREQ
        WORDS_FREQ = self.words_freq
        EDITS_FREQ = self.edits_freq
        self.WORDS = self.words_freq

    @staticmethod
    def load_freqs(path_to_freqs: str, sep: str = ' '):
        """
        Load frequencies from the file.
        :param sep: separator between word/edit and its frequency
        :param path_to_freqs: path to the file with frequencies
        :return: Counter with frequencies
        """

        with open(path_to_freqs, 'r', encoding='cp1251') as f:
            return Counter(
                {
                    pair.split(sep)[0]: int(pair.split(sep)[1])
                    for pair in f.readlines()
                }
            )

    @staticmethod
    def load_ngram_data(path: str):
        with open(path, 'r', encoding='cp1251') as f:
            return Counter(
                {
                    tuple(line.split()[1:]): int(line.split()[0])
                    for line in f.readlines()
                }
            )

    def compute_possible_prefixes(self):
        """
        Compute possible prefixes for the words in the vocabulary.
        :return: set of possible prefixes
        """

        return set(
            word[:i]
            for word in self.words_freq
            for i in range(len(word) + 1)
        )

    def correction(self, word: str):
        """Method for compatibility with Norvig's SpellChecker"""

        word = word.lower()

        return self.word_correction(word)

    def word_correction(self, word: str):
        """Most probable spelling correction for word."""

        all_candidates = self.candidates(word)

        if not all_candidates:
            return word

        word = max(all_candidates,
                   key=lambda word: self.word_prob(word) *
                                    self.edit_prob(all_candidates[word][0]) *
                                    (0.001 + all_candidates[word][1])
                   )

        return word

    def text_correction(self, text: str):
        """Most probable spelling correction for text."""

        # Tokenize the input text into words
        tokens = re.compile(r"(\w[\w']*\w|\w)").findall(text.lower().strip())
        
        # Correct the first word in the text
        corrected_text = [self.word_correction(tokens[0])]
        
        # Iterate over the rest of the words in the text
        for token in tokens[1:]:
            # Create a bigram with the last corrected word and the current word
            bigram = (corrected_text[-1], token)
        
            # If the bigram exists in the bigram frequency dataset, append the current word to the corrected text
            if bigram in self.bigrams_freq:
                corrected_text.append(token)
                continue
        
            # If the current word is a single character, append it to the corrected text
            if len(bigram[1]) == 1:
                corrected_text.append(token)
                continue
        
            # Generate possible corrections for the current word
            word_candidates = self.candidates(token)
        
            # If there are no candidates, append the current word to the corrected text
            if not word_candidates:
                corrected_text.append(token)
                continue
        
            # Create bigram candidates with the last corrected word and the word candidates
            bigram_candidates = [
                (bigram[0], word)
                for word in word_candidates
            ]
        
            # Select the bigram with the highest probability
            bigram = max(bigram_candidates, key=lambda bigram: self.bigram_prob(bigram))
        
            # Append the corrected word to the corrected text
            corrected_text.append(bigram[1])

        return ' '.join(corrected_text)

    def candidates(self, word: str):
        """Generate possible spelling corrections for word."""

        result = dict()

        generate_edits(head='', tail=word,
                       edit_distance=self.max_edit_distance, edits=[],
                       result=result, possible_prefixes=self.possible_prefixes)

        return result

    def word_prob(self, word: str):
        """Probability of `word`."""

        if word in self.words_freq:
            return self.words_freq[word] / sum(self.words_freq.values())

        # Default probability for unknown words
        return 1 / sum(self.words_freq.values())

    def edit_prob(self, edit: str):
        """Probability of `edit`."""
        avg_spell_error_freq = 0.05

        if not edit:
            return 1 - avg_spell_error_freq

        prob = avg_spell_error_freq
        for e in edit.split('+'):
            prob *= self.edits_freq.get(e, 1) / sum(self.edits_freq.values())

        return prob

    def bigram_prob(self, bigram: str):
        """Probability of `bigram`."""

        if bigram in self.bigrams_freq:
            return self.bigrams_freq[bigram] / (self.first_word_frequencies[bigram[0]] + 1)

        return 1 / (self.first_word_frequencies[bigram[0]] + 1)

    def calculate_first_word_frequencies(self):
        """Calculate frequencies of the first words in the bigrams."""

        first_word_frequencies = Counter()

        for bigram in self.bigrams_freq:
            first_word_frequencies[bigram[0]] += self.bigrams_freq[bigram]

        return first_word_frequencies

## Minimal testing while developing

In [3]:
spellchecker = CSSpellChecker(
    path_to_words_freq='data/internet/top_300k_words_freq.txt',
    word_sep='\t',

    path_to_edits_freq='data/internet/single_edits_freq.txt',
    edit_sep='\t',

    path_to_bigrams='data/coco/bigrams.txt',
    path_to_fivegrams='data/coco/fivegrams.txt',

    freq_threshold=700000
)

In [4]:
spellchecker.correction('helgo')

'hello'

In [5]:
spellchecker.text_correction("Helgo, my nama is cool. I al a studenn.")

'hello to name is cool i was a student'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

Justifications on implementation choices:
1. **N-gram dataset**: I used the COCA dataset for bigrams and fivegrams. The dataset is large and contains a lot of text data, which is important for the language model. The bigrams and fivegrams are used to calculate the probability of the word given the previous word. This is important for the context-sensitive spell correction.
2. **Word Frequencies**: I used the top 300k words from the internet. The words with the frequency less than 700k were removed. This is done to reduce the size of the vocabulary and speed up the computations. The words with the frequency less than 700k are considered rare and are not likely to be used in the context-sensitive spell correction. The dataset was composed of Google Web Trillion Word Corpus.
3. **Edit Frequencies**: I used the single edits frequencies from the internet. Those frequencies help me to calculate the probability of concrete sequence of edits.
4. **Edit Distance**: I used the edit distance of 2. This is a common choice for the spell correction tasks. The edit distance of 2 allows to correct the words with two typos.
5. **Bigram Probability**: The bigram probability is calculated as the frequency of the bigram divided by the frequency of the first word in the bigram. If the bigram is found in the dataset - it counts as correct. 

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [6]:
norvig_sc = NorvigSpellChecker(path_to_vocab='data/norvig/big.txt')

In [7]:
context_sensitive_sc = CSSpellChecker(
    path_to_words_freq='data/internet/top_300k_words_freq.txt',
    word_sep='\t',

    path_to_edits_freq='data/internet/single_edits_freq.txt',
    edit_sep='\t',

    path_to_bigrams='data/coco/bigrams.txt',
    path_to_fivegrams='data/coco/fivegrams.txt',

    freq_threshold=700000
)

### Tests written by hand

In [8]:
tests = {
    "Helgo, my nama is cool. I al a studenn.": "Hello, my name is cool. I am a student.",
    "Thiss sentennce hass a lot of missspelled worts.": "This sentence has a lot of misspelled worts."
}

In [9]:
for test, expected in tests.items():
    print("#" * 50)
    print()

    print(f"Test: {test}")
    print(f"Expected: {expected}")

    print("-" * 50)

    print(f"Norvig: {norvig_sc.text_correction(test)}")
    print(f"Context-sensitive: {context_sensitive_sc.text_correction(test)}")

    print()

##################################################

Test: Helgo, my nama is cool. I al a studenn.
Expected: Hello, my name is cool. I am a student.
--------------------------------------------------
Norvig: hello, by name in fool. a a a student.
Context-sensitive: hello to name is cool i was a student

##################################################

Test: Thiss sentennce hass a lot of missspelled worts.
Expected: This sentence has a lot of misspelled worts.
--------------------------------------------------
Norvig: hiss sentence has a not of misspelled words.
Context-sensitive: his sentence was a lot of missspelled forts


### Generating test set

In [10]:
test_text = ("No one is perfect and we do not claim to find every error in your text. "
             "That is just not possible with a machine-only check. If others claim they can do this automatically, it is just not correct."
             " The last resort is always a human (and even this person may fail from time to time). "
             "Nevertheless our online spellchecker will help you find most errors and will also make suggestions for grammatical improvements.")

In [11]:
test_text_cleared = ' '.join(re.compile(r"(\w[\w']*\w|\w)").findall(test_text.lower().strip()))

In [12]:
import random
import string


def misspell_word(word, p=0.1):
    misspelled_word = ""

    for i, c in enumerate(word):
        if random.random() < p:
            misspelled_word += random.choice(list(set(string.ascii_lowercase) - {c}) + [''])
        else:
            misspelled_word += c

    return misspelled_word


def misspell_text(text, word_noise=0.1, text_noise=0.1):
    tokens = re.compile(r"(\w[\w']*\w|\w)").findall(text.lower().strip())

    return ' '.join([misspell_word(token, word_noise) if random.random() < text_noise else token for token in tokens])

In [13]:
text_noises = [0.1, 0.3, 0.5, 0.7, 0.9]
word_noises = [0.1, 0.2, 0.3, 0.4]


def test_text_correction(text, word_noises, text_noises, verbose=False):
    if verbose:
        print("Initial clear text:", text, sep='\n')

    total_tests = len(text_noises) * len(word_noises)
    test_num = 0
    norvig_avg_acc = 0
    context_sensitive_avg_acc = 0
    for text_noise in text_noises:
        for word_noise in word_noises:
            print("#" * 50)
            print("Test number:", test_num + 1, f"out of {total_tests}")
            test_num += 1
            print()

            print("Text noise:", text_noise)
            print("Word noise:", word_noise)

            print("-" * 50)

            text_noised = misspell_text(text, word_noise, text_noise)

            norvig_correction = norvig_sc.text_correction(text_noised)
            context_sensitive_correction = context_sensitive_sc.text_correction(text_noised)

            norvig_acc = sum([1 for a, b in zip(norvig_correction.split(), test_text_cleared.split()) if a == b]) / len(
                test_text_cleared.split())
            context_sensitive_acc = sum(
                [1 for a, b in zip(context_sensitive_correction.split(), test_text_cleared.split()) if a == b]) / len(
                test_text_cleared.split())

            norvig_avg_acc += norvig_acc
            context_sensitive_avg_acc += context_sensitive_acc

            if verbose:
                print(f"Test:", text_noised, sep='\n')

                print("-" * 50)

                print(f"Norvig:", norvig_correction, sep='\n')
                print()
                print(f"Context-sensitive:", context_sensitive_correction, sep='\n')
                
                print()

            print("Norvig accuracy:", norvig_acc)
            print("Context-sensitive accuracy:", context_sensitive_acc)

            print()

    norvig_avg_acc /= total_tests
    context_sensitive_avg_acc /= total_tests

    print("#" * 50)
    print("Average accuracies:")
    print("Norvig:", norvig_avg_acc)
    print("Context-sensitive:", context_sensitive_avg_acc)

In [14]:
test_text_correction(test_text_cleared, word_noises, text_noises, verbose=True)

Initial clear text:
no one is perfect and we do not claim to find every error in your text that is just not possible with a machine only check if others claim they can do this automatically it is just not correct the last resort is always a human and even this person may fail from time to time nevertheless our online spellchecker will help you find most errors and will also make suggestions for grammatical improvements
##################################################
Test number: 1 out of 20

Text noise: 0.1
Word noise: 0.1
--------------------------------------------------
Test:
no one is perfect and we do not claim to find every error in your text that is just not possible with a machine only check if others claim they can do this autozaticalsy it is just not correct the last resort is always a human and even this person may fail from time to time nevertheless our online spellchecker will help you find most errors and will also make suggestions for grammatical improvements
--------

#### The context sensitive spell checker shows better results than the Norvig's spell checker. The average accuracy of the context sensitive spell checker is higher than the accuracy of the Norvig's spell checker. 